In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import matplotlib.pyplot as plt
import cv2
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from numpy import save
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from urllib import request,error
from urllib.parse import urlparse
import json
import math
import sys
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,datasets
import PIL
from PIL import Image
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.metrics import accuracy_score
import csv 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
driveroot = "drive/MyDrive/COMP9417/dataset/train"
cleanroot = os.path.join(driveroot,"clean")
clustered = os.path.join(driveroot,"transformed","clustered")
raw = os.path.join(driveroot,"transformed","raw")

In [ ]:
k = 4

In [ ]:
X_train=np.load(os.path.join(clustered,f"kmeans={k}_train.npy"))
y_train=np.load(os.path.join(raw,f"y_train.npy"))
X_val=np.load(os.path.join(clustered,f"kmeans={k}_val.npy"))
y_val=np.load(os.path.join(raw,f"y_val.npy"))
X_test=np.load(os.path.join(clustered,f"kmeans={k}_test.npy"))
y_test=np.load(os.path.join(raw,f"y_test.npy"))

In [ ]:
# reshape X_train to 2D
nsamples, nx, ny = X_train.shape
X_train = X_train.reshape((nsamples,nx*ny))
# reshape X_test to 2D
nsamples, nx, ny = X_test.shape
X_test = X_test.reshape((nsamples,nx*ny))
# reshape y_train to 2D
nsamples = y_train.shape[0]
y_train = y_train.reshape((nsamples,1))
# reshape y_test to 2D
nsamples = y_val.shape[0]
y_val = y_val.reshape((nsamples,1))
# reshape X_validation to 2D
nsamples, nx, ny = X_val.shape
X_val = X_val.reshape((nsamples,nx*ny))

In [ ]:
X_train = np.vstack((X_train, X_val))
y_train = np.vstack((y_train, y_val))
y_train = y_train.reshape((y_train.shape[0], ))

In [ ]:
optim = ["sgd", "adam"]                
activator = ["relu","logistic"]           
lr = [0.001, 0.01]                      
ep = [20]                 
batchsize = [16, 32]                 
dropout = [0, 0.5]                   
dense = [[16*5*128,640,64],[16*5*128,64],[10240],[640],[64]]           
parameter_grid = {
    'solver': optim,
    'activation': activator,
    'learning_rate_init': lr,
    'max_iter': ep,
    'batch_size': batchsize,
    'hidden_layer_sizes': dense
}

In [ ]:
mlp_clf = MLPClassifier()
CV_rnd_clf = RandomizedSearchCV(estimator=mlp_clf, param_distributions=parameter_grid, cv=4, n_jobs=-1,verbose=10)
CV_rnd_clf.fit(X_train, y_train)

In [ ]:
cv_results = CV_rnd_clf.cv_results_

In [ ]:
cv_re = []

for i in range(0, len(cv_results['params'])):
    cv_re.append(
        {
        'mean_fit_time': cv_results['mean_fit_time'][i],
        'std_fit_time':cv_results['std_fit_time'][i],
        'mean_score_time':cv_results['mean_score_time'][i],
        'std_score_time':cv_results['std_score_time'][i],
        'param_activation':cv_results['param_activation'][i],
        'param_batch_size':cv_results['param_batch_size'][i],
        'param_hidden_layer_sizes':cv_results['param_hidden_layer_sizes'][i],
        'param_learning_rate_init':cv_results['param_learning_rate_init'][i],
        'param_max_iter':cv_results['param_max_iter'][i],
        'param_solver':cv_results['param_solver'][i],
        'split0_test_score':cv_results['split0_test_score'][i],
        'split1_test_score':cv_results['split1_test_score'][i],
        'split2_test_score':cv_results['split2_test_score'][i],
        'split3_test_score':cv_results['split3_test_score'][i],
         'mean_test_score':cv_results['mean_test_score'][i],
         'std_test_score':cv_results['std_test_score'][i],
         'rank_test_score':cv_results['rank_test_score'][i],
        }
    )

In [ ]:
field_names = ['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_activation', 'param_batch_size', 'param_hidden_layer_sizes', 'param_learning_rate_init', 'param_max_iter', 'param_solver', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score']
with open(os.path.join(driveroot,f'MLP_CVresults_k={k}.csv'), 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(cv_re)